In [ ]:
#Imports
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Comment this lines if you have this stuff already installed
#!(yes |pip install geopandas)
#!(yes |pip install descartes)
#!(yes |conda install -c conda-forge geoplot)
#!(yes | pip install plotly)
import plotly.express as px
#!pip install spacy
#!python -m spacy download en_core_web_sm
#nltk.download('wordnet')
#nltk.download('punkt')
import nltk
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import spacy
import en_core_web_sm
#!pip install gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import KeyedVectors
from gensim import models
import gensim
%config IPCompleter.greedy=True
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs
from sklearn.decomposition import PCA
from matplotlib import pyplot
import warnings
warnings.filterwarnings('ignore')
from numpy import linalg as LA
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.integrate import odeint
from sklearn.model_selection import train_test_split
#import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from gensim import models
from gensim.models import Word2Vec


In [ ]:
#Gets the hashtag or user
def getter(text, char):
    pos = text.find(char)
    text = text[pos:]
    #Some users or hashtags finish with : or .
    if text.endswith(':') or text.endswith('.'):
        text = text[:-1]
    return text
def validUser(userName):
    if '@' in userName:
        user = getter(userName, '@')
        length = len(user)
        if (length > 1 and length <= 16):
            for char in user[1:]:
                if not(char.isalnum() or char == '_'): return False
            return True
    return False
def validLink(link):
    type1 = 'https://'
    type2 = 'http://'
    if type1 in link and len(link) > 9: return True
    if type2 in link and len(link) > 8: return True
    return False
def validHashtag(hashtag):
    if '#' in hashtag:
        hashtag = getter(hashtag, '#')
        hashtag = hashtag[1:]
        return hashtag.isalnum()
    return False

In [ ]:
tweetsInfo = pd.read_csv('./ToChangeKeywordsAndLocations/withoutEncoding.csv')
def analyzeTweets(text):
    result = [0,0,0] #number of usersTagged, hashtags and links
    text = text.split()
    for word in text:
        if validUser(word): result[0] += 1
        elif validHashtag(word): result[1] += 1
        elif validLink(word): result[2] += 1
    return result
def getInfo(dataList, pos):
    return dataList[pos]
#Aux column to get the result
tweetsInfo['aux_column'] = tweetsInfo.text.apply(analyzeTweets)

tweetsInfo['users_tagged'] = tweetsInfo.aux_column.apply(getInfo,args=(0,))
tweetsInfo['hashtags'] = tweetsInfo.aux_column.apply(getInfo,args=(1,))
tweetsInfo['links'] = tweetsInfo.aux_column.apply(getInfo,args=(2,))

del tweetsInfo['aux_column']
tweetsInfo.head()

In [ ]:
def validLocation(location):
    return int(location != 'unknown')
def sizes(size):
    if size == 'small': return 0
    if size == 'medium': return 5
    return 10
def tweetSize(tweetLength, minValue, intervalRange):
    if tweetLength < (minValue + intervalRange): return 'small'
    if (minValue + intervalRange) <= tweetLength and tweetLength < (minValue + 2 * intervalRange): return 'medium'
    return 'large'

tweetsInfo['tweet_length'] = tweetsInfo.text.str.len()
#Parallel coordinates to show the characteristics of the tweets
#First we get the range of each interval
minValue = tweetsInfo.tweet_length.min()
maxValue = tweetsInfo.tweet_length.max()
intervalRange = (maxValue - minValue) // 3

#We add a new column
tweetsInfo['tweet_size'] = tweetsInfo.tweet_length.apply(tweetSize, args = (minValue, intervalRange))

disastersDf = tweetsInfo
disastersDf.head()


In [ ]:
disastersDf['Valid_location'] = disastersDf.location.apply(validLocation)
disastersDf['tweet_size'] = disastersDf.tweet_size.apply(sizes)
disastersDf.head()

In [ ]:
X, y = disastersDf.iloc[:,5:], disastersDf.iloc[:,4]  #X tiene que tener todos los features distintos al target

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
trueTweets = disastersDf[disastersDf.target == 1]
falseTweets = disastersDf[disastersDf.target == 0]
falseTexts = falseTweets.text.to_list()
trueTexts = trueTweets.text.to_list()
falseSentences = [text.split() for text in falseTexts]
trueSentences = [text.split() for text in trueTexts]
falseModel = Word2Vec(falseSentences, min_count = 25)
trueModel = Word2Vec(trueSentences, min_count = 25)
print(falseModel)
words = list(falseModel.wv.vocab)
print(words)

In [ ]:
disastersDf['Total_elements'] = disastersDf.hashtags + disastersDf.users_tagged + disastersDf.links
disastersDf['links_hash'] = disastersDf.hashtags + disastersDf.links
disastersDf['links_users'] = disastersDf.users_tagged + disastersDf.links
disastersDf['hash_users'] = disastersDf.hashtags + disastersDf.users_tagged
train = disastersDf.iloc[:, 4:]
train

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
file = 'GoogleNews-vectors-negative300.bin.gz'

model = KeyedVectors.load_word2vec_format (file, binary=True)
model.wv.similar_by_word('mutation')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def get_average_vector(text):
    tokens = [w.lower() for w in word_tokenize(text) if w.lower() not in stop_words]
    return np.mean(np.array([model.wv[w] for w in tokens if w in model]), axis=0)

model.wv.similar_by_vector(get_average_vector(disastersDf.loc[0, 'text']))

In [ ]:
help(model)

In [ ]:
vector_list = [model[word] for word in words if word in model.vocab]

# Create a list of the words corresponding to these vectors
words_filtered = [word for word in words if word in model.vocab]

# Zip the words together with their vector representations
word_vec_zip = zip(words_filtered, vector_list)

# Cast to a dict so we can turn it into a DataFrame
word_vec_dict = dict(word_vec_zip)
df = pd.DataFrame.from_dict(word_vec_dict, orient='index')

In [ ]:
model.get_vector('fill').sum()

In [ ]:
from nltk.corpus import wordnet
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
    def replace(self, word):
        if wordnet.synsets(word):
            return word
        repl_word = self.repeat_regexp.sub(self.repl, word)
        
        if repl_word != word:
            return self.replace(repl_word)

        return repl_word

In [ ]:
import re
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'I\'m', 'I am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')
]
class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s

In [ ]:
def cleanTweet(tweet): #Modificado para sacar solo los links
    return ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
import string
def deletePunctuation(tokenizedText):
    x = re.compile('[%s]' % re.escape(string.punctuation))
    tokenized_text_no_punctuation = []
    for token in tokenizedText: #Agarro las palabras de la lista
        newToken = x.sub(u'', token)
        if not newToken == u'':
            tokenized_text_no_punctuation.append(newToken)
    return tokenized_text_no_punctuation
def deleteStopwords(tokenizedText, stopwords):
    return [word for word in tokenizedText if word not in stopwords]
def editText(text, stopwords, replacer, repeatReplacer):
    #Primero elimino los links
    text = cleanTweet(text)
    
    #Paso a lower el text
    text = text.lower()
    
    #Reemplazo los I'm por I am
    text = replacer.replace(text)
    
    #Elimino los caracteres repetidos, ej: ohhh por oh
    words = text.split()
    text = ' '.join(repeatReplacer.replace(word) for word in words)
    
    #Tokenizo el texto
    tokenizedText = nltk.word_tokenize(text)
    
    #Elimno los signos de puntuacion
    tokenizedText = deletePunctuation(tokenizedText)
    
    #Elimino los stopwords
    tokenizedText = deleteStopwords(tokenizedText, stopwords)
    
    editText = ' '.join(tokenizedText)
    return editText


In [ ]:
repeatReplacer = RepeatReplacer()
replacer = RegexpReplacer()
stop = set(stopwords.words('english'))

disastersDf['text'] = disastersDf.text.apply(editText, args = (stop, replacer, repeatReplacer))
disastersDf

In [ ]:
disastersDf['prom'] = disastersDf.text.apply(lambda x: sum([model.get_vector(i).sum() for i in x.split() if i in model.vocab]))

In [ ]:
disastersDf['prom'].apply(np.round).value_counts()